# TODO：概要

# ライブラリのインポート

In [49]:
import os
import time
import json
import datetime
import zoneinfo

import requests

from dotenv import load_dotenv, find_dotenv

from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.agents.models import (
    MessageTextContent,
    ListSortOrder,
    McpTool,
    MCPToolDefinition,
    RequiredMcpToolCall,
    SubmitToolApprovalAction,
    ToolApproval,
    CodeInterpreterTool,
    FunctionTool,
    ToolSet,
)

# 環境変数の取得

In [50]:
load_dotenv(override=True)

PROJECT_ENDPOINT=os.getenv("PROJECT_ENDPOINT")
AZURE_DEPLOYMENT_NAME=os.getenv("AZURE_DEPLOYMENT_NAME")

# クライアントの初期化

In [51]:
# AI Project Client を初期化
project_client = AIProjectClient(
    endpoint=PROJECT_ENDPOINT,
    credential=DefaultAzureCredential()
)

# AgentClient の作成
agents_client = project_client.agents

# ユーティリティ関数

In [52]:
def agent_run_outputs(thread_id, agents_client, target_dir="./output_images"):
    """
    指定したスレッドIDのRun実行結果（テキスト・画像）をNotebook上に表示＆画像は保存。
    """
    messages = agents_client.messages.list(thread_id=thread_id, order=ListSortOrder.ASCENDING)
    os.makedirs(target_dir, exist_ok=True)

    for message in messages:
        # テキスト出力
        if message.text_messages:
            for txt in message.text_messages:
                print(f"{message.role.upper()}: {txt.text.value}")
        
        # 画像出力
        if hasattr(message, "image_contents"):
            for image_content in message.image_contents:
                file_id = image_content.image_file.file_id
                file_name = f"{file_id}_image_file.png"

                agents_client.files.save(
                    file_id=file_id,
                    file_name=file_name,
                    target_dir=target_dir
                )
                print(f"Saved image: {file_name}")
                display(Image(filename=f"{target_dir}/{file_name}"))

# ツールの定義

In [ ]:
mcp_tool = McpTool(
    server_label="MicrosoftDocs",
    server_url="https://learn.microsoft.com/api/mcp",
)

# MCP ツール実行時の承認モードを never に設定
mcp_tool.set_approval_mode("never")

print(mcp_tool.resources)

{'mcp': [{'server_label': 'MicrosoftDocs', 'headers': {}, 'require_approval': 'never'}]}


# エージェントの作成

In [119]:
mcp_agent = agents_client.create_agent(
    model=AZURE_DEPLOYMENT_NAME,
    name="mcp_agent",
    instructions=(
        "あなたは、MCPツールを使用してユーザーを支援できる有用なエージェントです。"
        "利用可能なMCPツールを使用して、質問に答えてタスクを実行します。"    
    ),
    description=(
        "あなたは Microsoft の公式ドキュメントを検索できる MCP ツールを持っているエージェントです。"
    ),
    tools=mcp_tool.definitions,
    tool_resources=mcp_tool.resources,
)
print(f"Created Agent. AGENT_ID: {mcp_agent.id}")


Created Agent. AGENT_ID: asst_46kuHDRuUpeFnNDrWuVOkypt


In [120]:
agent_dict = mcp_agent.as_dict()
print(json.dumps(agent_dict, indent=2, ensure_ascii=False))

{
  "id": "asst_46kuHDRuUpeFnNDrWuVOkypt",
  "object": "assistant",
  "created_at": 1753607349,
  "name": "mcp_agent",
  "description": "あなたは Microsoft の公式ドキュメントを検索できる MCP ツールを持っているエージェントです。",
  "model": "gpt-4.1",
  "instructions": "あなたは、MCPツールを使用してユーザーを支援できる有用なエージェントです。利用可能なMCPツールを使用して、質問に答えてタスクを実行します。",
  "tools": [
    {
      "type": "mcp",
      "server_label": "MicrosoftDocs",
      "server_url": "https://learn.microsoft.com/api/mcp",
      "allowed_tools": null
    }
  ],
  "top_p": 1.0,
  "temperature": 1.0,
  "tool_resources": {},
  "metadata": {},
  "response_format": "auto"
}


# スレッドの作成

In [111]:
# Thread の作成
thread = agents_client.threads.create()
print(f"Created Thread. THREAD_ID: {thread.id}")

Created Thread. THREAD_ID: thread_rriOLxS2SimRUF5B90O7Ux1k


# ユーザーメッセージの追加

In [112]:
# メッセージの追加
user_message = "Azure AI Foundry Agent Service の最新情報を教えてください。"

message = agents_client.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_message,
)

print(f"Added Message. MESSAGE_ID: {message.id}")

Added Message. MESSAGE_ID: msg_UPYi1M4hNuiz6K64e6cdwFLj


# Run の実行

## MCP ツール自動承認で実行

In [ ]:
run = agents_client.runs.create_and_process(
    thread_id=thread.id,
    agent_id=mcp_agent.id,
    # MCP ツールのリソース情報を渡す（つまり、現状承認モード設定やリクエストヘッダーの設定はここで定義される）
    tool_resources=mcp_tool.resources 
)

if run.status == "failed":
    print(f"Run failed: {run.last_error}")
else:
    agent_run_outputs(thread.id, agents_client)

USER: Azure AI Foundry Agent Service の最新情報を教えてください。
ASSISTANT: Azure AI Foundry Agent Serviceの最新情報（2025年6月現在）の主な内容は以下の通りです。

### 直近の主なアップデート

#### 2025年6月
- **Deep Research Tool**  
  Azure OpenAIの `o3-deep-research` モデルとBing検索を利用したマルチステップリサーチツールが追加。

- **Model Context Protocol (MCP) Tool**  
  エージェントをリモートのModel Context Protocol (MCP) サーバーに接続できるようになりました。これにより、外部ツールとの連携が強化。

#### 2025年5月
- **一般提供（GA）開始**
  - Azure AI Foundry Agent Serviceが正式リリースされ、以下の主要機能が追加・強化：
    - **AI Foundry Visual Studio Code拡張機能**  
      エージェントのデプロイや設定など、様々な操作がVSCodeから可能に。
    - **Connected Agents**  
      タスクごとのエージェントをプライマリエージェントとシームレスに連携でき、複雑なマルチエージェントシステムの構築が容易に。
    - **Trace Agents**  
      スレッド単位でエージェントの入力・出力・プロセスをトレースしデバッグが可能。
    - **Azure Logic Appsとの統合**  
      新しいイベント（メール・チケット等）発生時に自動でエージェントをトリガー可能に。
    - **新しいエージェント用ツール**  
      - Bing Custom Search  
      - Morningstarデータソースとの連携

#### 2025年4月
- **Azure Monitor連携**によるエージェントの各種メトリクス取得
- **BYO Thread Storage**  
  会話履歴等のデータを自前のAzure Cosmos DB上に保

## MCP ツール手動承認で実行

In [ ]:
run = agents_client.runs.create(
    thread_id=thread.id,
    agent_id=mcp_agent.id
)

while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(1)
    run = agents_client.runs.get(thread_id=thread.id, run_id=run.id)

    if run.status == "requires_action" and isinstance(run.required_action, SubmitToolApprovalAction):
        tool_calls = run.required_action.submit_tool_approval.tool_calls
        if not tool_calls:
            print("No tool calls provided - cancelling run")
            agents_client.runs.cancel(thread_id=thread.id, run_id=run.id)
            break

        tool_approvals = []
        for tool_call in tool_calls:
            if isinstance(tool_call, RequiredMcpToolCall):
                try:
                    print(f"Approving tool call: {tool_call}")
                    tool_approvals.append(
                        ToolApproval(
                            tool_call_id=tool_call.id,
                            approve=True, # 承認する場合は True、拒否する場合は False
                            headers=mcp_tool.headers,
                        )
                    )
                except Exception as e:
                    print(f"Error approving tool_call {tool_call.id}: {e}")

        print(f"tool_approvals: {tool_approvals}")
        if tool_approvals:
            # ツール承認を送信
            agents_client.runs.submit_tool_outputs(
                thread_id=thread.id,
                run_id=run.id,
                tool_approvals=tool_approvals
            )

    print(f"Current run status: {run.status}")

Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Approving tool call: {'id': 'call_QRkKwWfOqVxoeLpQJL8LflaA', 'type': 'mcp', 'arguments': '{"question":"Azure AI Foundry Agent Service ã\x81®æ\x9c\x80æ\x96°æ\x83\x85å\xa0±"}', 'name': 'microsoft_docs_search', 'server_label': 'MicrosoftDocs'}
tool_approvals: [{'tool_call_id': 'call_QRkKwWfOqVxoeLpQJL8LflaA', 'approve': True, 'headers': {}}]
Current run status: RunStatus.REQUIRES_ACTION
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.COMPLETED


In [ ]:
agent_run_outputs(thread.id, agents_client)

USER: Azure AI Foundry Agent Service の最新情報を教えてください。
ASSISTANT: Azure AI Foundry Agent Serviceの最新情報（2025年6月時点）は以下の通りです。

### 最新リリース・アップデート抜粋

#### 2025年6月の主な新機能
- **Deep Research tool**  
  Azure OpenAIの新しい`o3-deep-research`モデルとBing Searchを活用したマルチステップリサーチツールを利用可能。
  - [Deep Research tool（公式ドキュメント）](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/deep-research)

- **Model Context Protocol（MCP）tool**  
  MCPサーバー上のツールと連携できる機能が拡張。外部のModel Context Protocolのツールをエージェントと接続可能になり、AIエージェントの拡張性が向上。
  - [MCP tool（公式ドキュメント）](https://learn.microsoft.com/en-us/azure/ai-foundry/agents/how-to/tools/model-context-protocol)

#### 2025年5月のハイライト
- **サービスのGA（General Availability）**  
  Azure AI Foundry Agent Serviceが一般提供開始に。

- **Visual Studio Code拡張**  
  エージェントのデプロイや設定作業がVS Codeから直接可能。

- **Connected Agents**  
  主体エージェントとタスク特化エージェントが連携し、マルチエージェントシステムを外部オーケストレータ不要で構築可能。

- **エージェントのトレース・監視**  
  個々のエージェント実行に関与するプロセスの入出力や順序を可視化・デバッグ。

- **Azure Logic Appsによる自動トリガー**  
  イベント（例：メール受信や新規

※ ここで、Azure AI Foundry 上の Web UI からエージェントのトレースを確認してみよう。

# Agent ID を .env ファイルに保存
※ 今回作成したエージェントを、後続の Connected Agents のハンズオン演習で使用するため永続化します。

In [114]:
# 変数の定義
agent_env_key = "FOUNDRY_MCP_AGENT_ID"
agent_env_value = mcp_agent.id

# .envファイルのパスを自動探索
env_path = find_dotenv()  # 見つからなければ''を返す
if not env_path:
    raise FileNotFoundError(".envファイルが見つかりませんでした。")

# AGENT_ID を .env ファイルに追記
with open(env_path, "a", encoding="utf-8") as f:
    f.write(f'\n{agent_env_key}="{agent_env_value}"')

print(f'.envファイルに {agent_env_key}=\"{agent_env_value}\" を追記しました。')

# .envの自動読み込み（以降os.environで参照可能）
load_dotenv(env_path)


.envファイルに FOUNDRY_MCP_AGENT_ID="asst_fnOAafS1zThfjtAClbEBUb0m" を追記しました。


True